<a href="https://colab.research.google.com/github/Justin21523/edge-deid-studio/blob/feature%2Fadd-gpt2-onnx-export-notebook/notebooks/05_export_onnx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 一、環境準備（Cell 1–3）

In [ ]:
!pip install -q transformers onnx onnxruntime onnxruntime-tools accelerate

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.environ['HF_HOME']         = '/content/drive/MyDrive/hf_cache'
os.environ['TRANSFORMERS_CACHE'] = '/content/drive/MyDrive/hf_cache/transformers'
os.makedirs(os.environ['TRANSFORMERS_CACHE'], exist_ok=True)

In [ ]:
import torch
print("GPU available:", torch.cuda.is_available())

> **為什麼**：
>
> 1. 安裝 `onnx` 與 `onnxruntime`、`onnxruntime-tools` 來支援 ONNX 格式的匯出與驗證。
> 2. 確保所有 Hugging Face 的快取都放到 Drive，不會因為重啟 Colab 而遺失。
> 3. 確認 GPU 狀態，雖然匯出 ONNX 可以跑在 CPU，但後面若要做量化、轉 Qualcomm DLC 時可能需要 GPU。

## 二、載入微調後的 GPT-2（Cell 4）

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_dir = "models/gpt2/v1.0"       # 與 03_finetune_gpt2.ipynb 的 output_dir 相同
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model     = AutoModelForCausalLM.from_pretrained(model_dir)
model.eval()                         # 切到推理模式，關閉 dropout

> **為什麼**：
> 要把剛才 fine-tune 完的模型載入，才能把它轉成 ONNX。



## 三、匯出 ONNX（Cell 5）

In [ ]:
import os
import torch

# 1. 準備輸出目錄
onnx_path = "edge_models/gpt2/v1.0/gpt2.onnx"
os.makedirs(os.path.dirname(onnx_path), exist_ok=True)

# 2. 隨機 sample 一組 token 做為範例輸入
inputs = tokenizer("Hello world", return_tensors="pt")

# 3. ONNX export（在有 GPU 時可同樣執行在 CPU）
with torch.no_grad():
    torch.onnx.export(
        model,
        (inputs["input_ids"], inputs["attention_mask"]),
        onnx_path,
        opset_version=13,
        do_constant_folding=True,
        input_names=["input_ids", "attention_mask"],
        output_names=["logits"],
        dynamic_axes={
            "input_ids": {0: "batch_size", 1: "seq_len"},
            "attention_mask": {0: "batch_size", 1: "seq_len"},
            "logits": {0: "batch_size", 1: "seq_len"}
        }
    )
print("✅ ONNX model saved to", onnx_path)

> **原理**：
>
> * `torch.onnx.export` 會把 PyTorch 模型 graph 轉成 ONNX 格式。
> * `opset_version=13` 是較新的 ONNX 版本，支援 Transformer 常用算子。
> * `dynamic_axes` 允許 batch size、序列長度在推理時動態改變。
> * `do_constant_folding=True` 會把 graph 中可合併的常數運算先做掉，加速後續推理。

## 四、動態量化 (Int8 Quantization)（Cell 6）

In [ ]:
from onnxruntime.quantization import quantize_dynamic, QuantType

onnx_int8 = onnx_path.replace(".onnx", "_int8.onnx")
quantize_dynamic(
    onnx_path,
    onnx_int8,
    weight_type=QuantType.QInt8
)
print("✅ Quantized ONNX model saved to", onnx_int8)

> **為什麼**：
>
> * Int8 量化可大幅減少模型大小與記憶體需求，適合邊緣裝置。
> * `quantize_dynamic` 只對權重做量化，不影響原本 graph 結構，速度與精度通常平衡良好。

## 五、轉成 Qualcomm Edge (DLC) 格式（Cell 7）

In [ ]:
# 如果你已安裝 Qualcomm SNPE SDK，就可以用以下指令轉 DLC：
# snpe-onnx-to-dlc \
#   --input_network edge_models/gpt2/v1.0/gpt2_int8.onnx \
#   --output_network edge_models/gpt2/v1.0/gpt2.dlc \
#   --input_dim input_ids:1,128 \
#   --input_dim attention_mask:1,128

# 轉完之後，你就會在 edge_models/gpt2/v1.0/ 底下看到 gpt2.dlc

> **原理**：
>
> * Qualcomm 的 SNPE 工具能把 ONNX 轉成自家專用的 `.dlc` 格式，方便部署到 Snapdragon NPU。
> * `--input_dim` 用來硬編輸入 tensor shape，SNPE 需要知道最大維度才能做優化。

## 六、小結（Cell 8）

In [ ]:
print("🚀 Export pipeline complete!")
print("Check the following files under edge_models/gpt2/v1.0/:")
print("  • gpt2.onnx")
print("  • gpt2_int8.onnx")
print("  • gpt2.dlc   (如果已執行 SNPE 轉檔)")

### 完整流程回顧

1. **安裝環境**：Transformers + ONNX 工具
2. **載入模型**：從 `models/gpt2/v1.0` 讀入微調後權重
3. **匯出 ONNX**：`torch.onnx.export` + `dynamic_axes`
4. **量化**：將 ONNX 權重轉為 Int8，減少模型大小
5. **Qualcomm DLC**：（選做）使用 SNPE SDK 把量化後的 ONNX 轉成 `.dlc`